In [2]:
import re
import requests
import json
import time
import random
from serpapi import GoogleSearch

In [3]:
def eliminar_guiones(marca):
    return marca.replace("-", "").replace(" ", "")

In [4]:
def variaciones_separadores(marca):
    separadores = ["_", "-", ".", ""]
    variaciones = []
    for sep in separadores:
        variacion = marca.replace(" ", sep)
        variaciones.append(variacion)
    return variaciones

In [5]:
def variaciones_sin_separadores(marca):
    marca_sin_separadores = marca.replace("-", "").replace(" ", "")
    
    variaciones = [
        marca_sin_separadores.lower(),
        marca_sin_separadores.upper(), 
        marca_sin_separadores.title(), 
        marca_sin_separadores[0].upper() + marca_sin_separadores[1:].lower()
    ]
    
    return list(variaciones)
    
    

In [6]:
def variaciones_mayusculas(marca):
    return [marca.lower(), marca.upper(), marca.title(), marca[0].upper()+marca[1:].lower()]
    

In [7]:
def variaciones_geograficas(marca):
    geos = ["MX", "US"]
    variaciones = []
    for geo in geos:
        variaciones.append(f"{marca}{geo}")
        variaciones.append(f"{marca}_{geo}")
        variaciones.append(f"{marca}-{geo}")
    return variaciones

In [8]:
def variaciones_numericas(marca):
    numeros = [str(i) for i in range(1, 6)] + ["2020", "2021", "2022", "2023", "2024", "2025"]
    variaciones = []
    for num in numeros:
        variaciones.append(f"{marca}{num}")
        variaciones.append(f"{marca}_{num}")
        variaciones.append(f"{marca}-{num}")
    return variaciones

In [9]:
def variaciones_palabras_adicionales(marca):
    palabras = ["official", "fan", "page", "oficial", "america", "mexico"]
    variaciones = []
    for palabra in palabras:
        variaciones.append(f"{marca}{palabra}")
        variaciones.append(f"{marca}_{palabra}")
        variaciones.append(f"{marca}-{palabra}")
        variaciones.append(f"{marca} {palabra}")
    return variaciones

In [10]:
def variaciones_errores_tipograficos(marca):
    reemplazos = {
        "a": ["@", "4"],
        "e": ["3"],
        "i": ["1", "!"],
        "o": ["0"],
        "s": ["5", "$"]
    }
    variaciones = []
    for i, char in enumerate(marca):
        if char.lower() in reemplazos:
            for reemplazo in reemplazos[char.lower()]:
                nueva_variacion = marca[:i] + reemplazo + marca[i+1:]
                variaciones.append(nueva_variacion)
    return variaciones

In [11]:
def generar_todas_variaciones(marca):

    marca_sin_caracteres = eliminar_guiones(marca)
    marcas = [marca, marca_sin_caracteres]
    variaciones = []
    for m in marcas:
        
        variaciones.extend(variaciones_separadores(m))
        variaciones.extend(variaciones_mayusculas(m))
        variaciones.extend(variaciones_geograficas(m))
        variaciones.extend(variaciones_numericas(m))
        variaciones.extend(variaciones_palabras_adicionales(m))
        variaciones.extend(variaciones_errores_tipograficos(m))
        variaciones.extend(variaciones_sin_separadores(m))
    
    variaciones = list(set(variaciones))
    return variaciones

In [12]:
def generar_urls_con_variaciones(variaciones, plataformas):
    patrones_url = {
        "facebook": "https://www.facebook.com/{username}",
        "x": "https://x.com/{username}",
        "instagram": "https://www.instagram.com/{username}",
        "tiktok": "https://www.tiktok.com/{username}"
    }
    
    urls_por_plataforma = {plataforma: [] for plataforma in plataformas}
    
    for plataforma in plataformas:
        if plataforma in patrones_url:
            for variacion in variaciones:
                url = patrones_url[plataforma].format(username=variacion)
                urls_por_plataforma[plataforma].append(url)
    
    return urls_por_plataforma

In [22]:
# Lista de agentes de usuario para realizar solicitudes web con diferentes navegadores y sistemas operativos.
# Esto se usa para evitar bloqueos o restricciones al hacer múltiples peticiones a un mismo servidor.
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1",
    ]

In [23]:
# Esta función verifica si una URL es accesible haciendo una solicitud HTTP GET.
# COn esto descartamos enlaces rotos o inválidos antes de procesarlos en otras funciones.
def verificar_url(url):
    try:
        response = requests.get(url, timeout=10) # Se establece un tiempo límite de 10 segundos para evitar bloqueos.
        if response.status_code == 200: # Si el código de respuesta es 200, significa que la URL es válida.
            return True
        else:
            return False
    except requests.exceptions.RequestException as e:
        print(f"Error al verificar {url}: {e}")
        return False

In [24]:
# Realiza una búsqueda en Bing y extrae los enlaces de los resultados de búsqueda.
# Se usa Bing en lugar de Google debido a las restricciones de acceso directo a los resultados de búsqueda de Google.
def buscar_en_google(consulta, num_resultados=10):
    base_url = "https://www.bing.com/search"
    headers = {
        "User-Agent": random.choice(USER_AGENTS)
    }
    params = {"q": consulta, "num": num_resultados}
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        html = response.text
        enlaces = re.findall(r'/url\?q=(https?://[^&]*)', html)
        return enlaces[:num_resultados]
    except Exception as e:
        print(f"Error al realizar la búsqueda en Google: {e}")
        return []

In [ ]:
# Utiliza la API de SerpAPI para obtener resultados de Google sin restricciones.
# Esto permite obtener resultados de manera más confiable que el scraping manual... Esta pendiente de probar. solo se
# pueden hacer 100 requests por mes
def buscar_en_google_ser(consulta, num_resultados=5):
    params = {
        "q": consulta,
        "num": num_resultados,
        "api_key": "6c687ddff13803c53d2bd044e9905940fb42f5811d1bbd77fab8f9a6a5cbf6fa"
    }
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
        return [result["link"] for result in results.get("organic_results", [])]
    except Exception as e:
        print(f"Error al realizar la búsqueda en Google: {e}")
        return []

In [16]:
def simular_pagerank(resultados_totales):
    return sorted({url: len(set(enlaces)) for url, enlaces in resultados_totales.items()}.items(), key=lambda x: x[1], reverse=True)

In [19]:
# Guarda los resultados de verificación en un archivo JSON y realiza búsquedas si la URL existe.
# Se utiliza para almacenar los datos obtenidos y evitar hacer las mismas solicitudes repetidamente.
def resultados_json(nombre_marca, urls_generadas):
    archivo_resultados = f"resultados_{nombre_marca.lower().replace('-', '')}.json"
    try:
        with open(archivo_resultados, "r") as archivo:
            resultados_totales = json.load(archivo) # Carga datos previos para evitar procesar URLs repetidas.
            print(f"Archivo de resultados '{archivo_resultados}' cargado correctamente.") 
    except FileNotFoundError:
        print(f"Archivo de resultados '{archivo_resultados}' no encontrado. Se generarán nuevos resultados.")
        resultados_totales = {} # Si el archivo no existe, se crea un diccionario vacío.

    for plataforma, urls in urls_generadas.items():
        for url in urls:
            if url not in resultados_totales:
                print(f"Verificando existencia de: {url}")
                existe = verificar_url(url)
                if existe:
                    print(f"La URL {url} existe. Realizando búsqueda en Bing...")
                    resultados_totales[url] = buscar_en_google(url, num_resultados=5)
                else:
                    print(f"La URL {url} no existe.")
                    resultados_totales[url] = []  # Guardar una lista vacía si la URL no existe
                time.sleep(random.uniform(5, 10))  # Retraso aleatorio entre 5 y 10 segundos

    with open(archivo_resultados, "w") as archivo:
        json.dump(resultados_totales, archivo, indent=4)
    print(f"Resultados de verificación guardados en '{archivo_resultados}'")
    return resultados_totales

In [20]:
if __name__ == "__main__":
    nombre_marca = "coca-cola"
    plataformas = ["x"]
    
    # Generar variaciones
    variaciones = generar_todas_variaciones(nombre_marca)
    print(f"Variaciones generadas para '{nombre_marca}':")
    for i, variacion in enumerate(variaciones, 1):
        print(f"{i}. {variacion}")
    
    # Generar URLs
    urls_generadas = generar_urls_con_variaciones(variaciones, plataformas)
    for plataforma, urls in urls_generadas.items():
        print(f"\nURLs generadas para {plataforma}:")
        for i, url in enumerate(urls, 1):
            print(f"{i}. {url}")

    # Obtener y guardar resultados
    resultados_totales = resultados_json(nombre_marca, urls_generadas)

    # Simular PageRank
    pagerank = simular_pagerank(resultados_totales)
    print("\nResultados de PageRank:")
    for url, score in pagerank:
        print(f"{url}: {score} enlaces")

    umbral = 5
    cuentas_oficiales = [url for url, score in pagerank if score >= umbral]
    print("\nCuentas oficiales:")
    for cuenta in cuentas_oficiales:
        print(cuenta)

Variaciones generadas para 'coca-cola':
1. cocacola_official
2. Coca-cola
3. coca-cola-US
4. c0cacola
5. coca-cola-2
6. cocacolaamerica
7. coca-colafan
8. coca-cola2023
9. coca-cola_4
10. COCA-COLA
11. coca-cola page
12. coca-colaamerica
13. coc@-cola
14. cocacola_US
15. cocacol@
16. cocacolaoficial
17. cocacola-fan
18. coca-cola2020
19. cocacola-america
20. cocacola1
21. cocacola official
22. coca-cola_2024
23. cocacola2024
24. cocacolamexico
25. coca-cola america
26. cocacola2025
27. cocacola-3
28. coca-cola2021
29. cocacola-2021
30. cocacola_1
31. coca-cola_MX
32. coca-cola_fan
33. cocacola-oficial
34. coca-cola4
35. cocacola oficial
36. cocacola_2023
37. coca-cola_2023
38. coca-colaUS
39. cocacola2
40. cocacola_oficial
41. coca-col4
42. cocacola mexico
43. cocacola-page
44. cocacola_2025
45. coca-cola_5
46. coca-cola1
47. coca-cola2025
48. coca-cola_official
49. cocacola3
50. cocacola-official
51. coca-cola official
52. coca-cola
53. coca-cola-4
54. coca-cola_1
55. coca-cola_americ

In [ ]:
if __name__ == "__main__":
    nombre_marca = "pepsi"
    plataformas = ["x"]
    
    # Generar variaciones
    variaciones = generar_todas_variaciones(nombre_marca)
    print(f"Variaciones generadas para '{nombre_marca}':")
    for i, variacion in enumerate(variaciones, 1):
        print(f"{i}. {variacion}")
    
    # Generar URLs
    urls_generadas = generar_urls_con_variaciones(variaciones, plataformas)
    for plataforma, urls in urls_generadas.items():
        print(f"\nURLs generadas para {plataforma}:")
        for i, url in enumerate(urls, 1):
            print(f"{i}. {url}")

    # Obtener y guardar resultados
    resultados_totales = resultados_json(nombre_marca, urls_generadas)

    # Simular PageRank
    pagerank = simular_pagerank(resultados_totales)
    print("\nResultados de PageRank:")
    for url, score in pagerank:
        print(f"{url}: {score} enlaces")

    umbral = 5
    cuentas_oficiales = [url for url, score in pagerank if score >= umbral]
    print("\nCuentas oficiales:")
    for cuenta in cuentas_oficiales:
        print(cuenta)